## Collaborative Filtering

The collaborative filtering algorithm is simple; there are only a few things that need to be in the pipeline to produce recommendations. In each of the pipeline steps, you’ve a list of choices that affect the outcome. We’ll look at each step in detail to make sense of it all.

I am nearly surprised that Amazon hasn’t come up with something different by now. The algorithm is used to produce Amazon’s Recommended for You page. Mine is shown in figure 8.1. As you can see, I’ve bought books on statistics and Django. The overall idea is to find items that are rated similarly to the items already rated or bought.

![amazon-recs](img/amazon2.jpg)

You’ve certainly come across recommendation engines before when using an online shopping site like Macys or Amazon. Macys tracks the purchasing habits of all its shoppers, and when you log onto the site, it uses this information to suggest products you might like. Amazon can even suggest movies you might like, even if you’ve only bought items from it before. Some online concert ticket agencies will look at the history of shows you’ve seen before and alert you to upcoming shows that might be of interest. Sites like reddit.com let you vote on links to other web sites and then use your votes to suggest other links you might find interesting.

From these examples, you can see that preferences can be collected in many different ways. Sometimes the data are items that people have purchased, and opinions about these items might be represented as yes/no votes or as ratings from one to five. In this chapter, we’ll look at different ways of representing these cases so that they’ll all work with the same set of algorithms.

Though, most people consider themselves unique and don’t like to be segmented into a particular type. But that’s exactly what using collaborative filtering to calculate recommendations is all about. In all its simplicity, collaborative filtering recommends a list of items for you. The list is created based on people who like the same things as you, but who also like something that you haven’t yet consumed.

### Early examples

Around 1992 at the Xerox PARC (Palo Alto Research Center), when they realized that the number of emails sent had exploded, “...resulting in users being inundated by huge streams of incoming documents.”  I can’t help thinking that in 1992 they truly had seen nothing yet, but as in so many other cases, Xerox PARC was ahead of its time—maybe also in information overload.

The mail system they built was based on the assumption that you always have a few users who’d read everything immediately and then endorse those items of interest, while most users would read only what looked intriguing. The mail system was called Tapestry, a name you’ll often read in recommender system literature.

Two years later, the GroupLens project, a collaborative effort between MIT and the University of Minnesota, created “an open architecture for collaborative filtering of Netnews.” GroupLens (https://grouplens.org) wanted to solve the same problem of information overload and wanted to enable people to rate newsgroup messages. This time, the system was built on the assumption that people who previously agreed with the ratings were likely to agree with them again.

### Helping each other
The assumption on which collaborative filtering is based is that **together we can be better**, and together we’ll better understand each other. Sounds beautiful and a bit cheesy, like the ending of an epic Hollywood film, but this is the idea behind collaborative filtering. Also, you need to assume that people principally keep their tastes over time and that if you agreed with somebody in the past, you’ll likely agree with them in the future. Let’s try to be more concrete before we dig into the theory of collaborative filtering and how to calculate it.

In content based methods, you looked at recommendations that were based on what people had bought in the past by looking at their shopping baskets; now you’ll concentrate on the user. You could say that we’re asking the question, “If the user was a shopping cart what would be in it?”

Libraries sometimes have posters saying, “If you liked this popular book X, then you should also try this (maybe not so popular) book Y.” These posters are directed toward a large group of people and often work well; they’re similar to a filtered chart. Instead, what you want to do is to create individualized content lists or at least lists for small groups of like-minded users. You don’t want to print them out and hang them on your walls, but rather create and present them instantly when a user arrives at your website.

Neighborhood-based filtering can be handled in two ways. You could find users with similar film tastes as yours and then recommend films they’ve liked but that you haven’t seen; this is user-based filtering. Alternatively, you can find items similar to items that you already like; this is item-based filtering. Both similarities between users and items are calculated based on the ratings. 

![items_vs_users](img/item_vs_user.jpg)

## KNN

The collaborative filtering approach builds a model from a user’s past behaviors (items previously purchased or selected and/or numerical ratings given to those items) as well as similar decisions made by other users. This model is then used to predict items (or ratings for items) that the user may have an interest in.

In a real world setting, data collected from explicit feedbacks like movie ratings can be very sparse and data points are mostly collected from very popular items and highly engaged users. Large amount of less known items don’t have ratings at all. Let’s see plot the distribution of movie rating frequency.

![movie ratings](img/movie_rating_dist.png)

If we zoom in or plot it on a log scale, we can find out that only about 13,500 out of 58,098 movies received ratings by more than 100 users and the majority rest are much less known with little or no user-interactions. These sparse ratings are less predictable for most users and highly sensitive to an individual person who loves the obscure movie, which makes the pattern very noisy.

Most models make recommendations based on user rating patterns. To remove noisy pattern and avoid “memory error” due to large datasets, we may choose to filter our dataframe of ratings to only popular movies. After filtering, we are left with top or head movies in the Ratings data, which is enough for a recommendation model.

For today's audience, this can also be the case for implicit ratings when there is a large variety of items from men's suits to women's shoes to dinnerware.  

Collaborative filtering systems use the actions of users to recommend other movies. In general, they can either be user-based or item-based. The item based approach is usually preferred over user-based approach for several reasons. 

  1. User-based approach is often harder to scale because of the dynamic nature of users, often looking for different areas.
  2. Users constantly join the system, requiring constant retraining.  
  2. Items usually don’t change much in terms of how they relate to other items
  3. Item based approach often can be computed offline and served without constantly re-training.

To implement an **item based collaborative filtering**, KNN is a good go-to model and also a very good baseline for recommender system development. But what is the KNN? **KNN is a non-parametric, lazy learning method**. It uses a database in which the data points are separated into several clusters to make inference for new samples. 

KNN does not make any assumptions on the underlying data distribution but it relies on **item feature similarity**. When KNN makes inference about a movie, KNN will calculate the “distance” between the target movie and every other movie in its database, then it ranks its distances and returns the top K nearest neighbor movies as the most similar movie recommendations.

![knn](img/knn.png)

In [1]:
from scipy.sparse import csr_matrix

# pivot ratings into movie features
df_movie_features = df_ratings.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)

# convert dataframe of movie features to scipy sparse matrix for space
mat_movie_features = csr_matrix(df_movie_features.values)

NameError: name 'df_ratings' is not defined

Now our training data has a very high dimensionality. KNN’s performance will suffer from curse of dimensionality if it uses “euclidean distance” in its objective function. Euclidean distance is unhelpful in high dimensions because all vectors are almost equidistant to the search query vector (target movie’s features). Instead, we will use cosine similarity for nearest neighbor search. There is also another popular approach to handle nearest neighbor search in high dimensional data, [locality sensitive hashing](https://en.wikipedia.org/wiki/Locality-sensitive_hashing), which is powerful and computationaly efficient, but still can suffer from garbage in/garbage out.


Let's take a look at a knn recomender in practice.

Check out the [movie_recommender_using_KNN](recommenders/movie_recommender_using_KNN.ipynb) for a local implementation.  For an exercise try implementing the KNN on emlp.  Then see if you can improve the algorithm!  If you need help with EMLP, see the emlp-labs-macys folder for useful examples from the last class.  

### COLD-START PROBLEMS

Collaborative filtering requires data, which is a problem when you recommend for a successful major retailer and receive new many users as well as constantly adding new items of the latest fashions—you have no data for generating recommendations. One way to get around this dilemma is to ask new users to rate a few items when they arrive. Twitter uses this approach to start a new user's feed.  Alternatively, it’s a good idea to create a new arrivals list to showcase new items because many users like to check out new products.  Another way could be to use exploit/explore methods such as those we will use in multi-armed bandits.  Another way to integrate new items is to rely on their content features, such as brands and styles.  

## Factorizing and Latent Factors

Most feedback in large systems is implcit.  We watch users and see what they do.  We assume that they are doing something that generally they like and take that as a positive interest.  This generates much more data than getting explicit feedback.  However we still need to deal with the fact that with tens of thousands of items most users have never engaged with most items.  Our search for ways to estimate the likelihood that a user will like a never before seen item leads us to search for latent factors.

Latent factor recommenders are a relatively new discovery, getting their real breakthrough when the Netflix Prize competition. The winner was an ensemble recommender algorithm, which means mixing many different algorithms to produce the final result (and, incidentally, the topic of chapter 12). The winning ensemble was so complicated that it never went into production. Instead, another solution made by Simon Funk became famous for getting close to winning because he blogged about it. His finding has been the basis for many other solutions since.  

Finding latent factors is a task that can be done in many ways. In the scope of collaborative filtering, finding latent factors has been done primarily with matrix factorizations based on the rating matrix.

A reason to reduce the dimensions could be to extract a signal from the data. We will use that signal to make predictions.  

In a sense, you could have the same information in the points as in the line as shown in the figure, only the points have noise too. The same principle applies when you do dimension reduction, where you have high-dimensional data.

If R has n rows and m columns (as in n users and m items), you call it an n × m matrix (read n by m); the size of U will be an n × d matrix and V is a d × m matrix. If you look at the matrix shown previously, you’ll have a formula as follows:

![uv](img/uv.jpg)

This is called a UV-decomposition. Here you set the d to be 2, but it could also have been 3 or 4 or even 5 (since the original matrix is 6 × 6). The idea is that you want to decompose the matrix R into hidden features (read columns for users and rows for items) for items and for users. In the field of recommender systems, you’d call the U the user-feature matrix and the V the item-feature matrix



## ALS

Alternating least squares is one way to break up a matrix into 2 approximating matrices.  It is not the only way.  On a single machine it is not generally the best way.  But it does work well on a cluster and in distributed computing.  

In general splitting up matrices into two or more maller matrices is called NMF, or non-negative matrix factorization.  It is an area of active reasearch and development and has been for decades.  

If you are interested in learning more about matrix factorizing, I recommend this great [tutorial](https://nbviewer.jupyter.org/github/fastai/numerical-linear-algebra/blob/master/nbs/2.%20Topic%20Modeling%20with%20NMF%20and%20SVD.ipynb#Singular-Value-Decomposition-(SVD))

Let's dig into the math of ALS decomposition and distributed ALS.  

Whiteboard to Review some math... 




# Implicit Alternating Least Squares
Recommender systems frequently use matrix factorization models to generate personalized recommendations for users. These models have been found to work well on recommending items, and can be easily reused for calculating related artists.

Many of the MF models used in recommender systems assume explicit data, where the user has rated both things they like and dislike using something like a 5 star rating scale. They typically work by treating the missing data as an unknown, and then minimizing the reconstruction error using SGD.

User web data here is implicit though - we can assume that a user listening to an artist means they like it, but we don't have the corresponding signal that a user doesn't like an artist. Implicit data is usually more plentiful and easier to collect than explicit data - and even when you have the user give 5 star ratings the vast majority of those ratings are going to be positive only so you need to account for implicit behaviour anyways.

This means we can't just treat the missing data as unknowns, instead we have to treat a user not listening to an artist as being a signal that the user might not like that artist.

This presents a couple of challenges in learning a factorized representation.

The first challenge is in doing this factorization efficiently: by treating the unknowns as negatives, the naive implementation would look at every single entry in our input matrix. Since the dimensionality here is roughly 360K by 300K - there are over 100 billion total entries to consider, compared to only 17 million non zero entries.

The second problem is that we can't be certain that a user not listening to an artist actually means that they don't like it. There could be other reasons for the artist not being listened to, especially considering that we only have the top 50 most played artists for each user in the dataset.

The Collaborative Filtering for Implicit Feedback Datasets paper accounts for both of these challenges in an elegant way.

To handle the case where we're not confident about our negative data, this approach learns a factorized matrix representation using different confidence levels on binary preferences: unseen items are treated as negative with a low confidence, where present items are treated as positive with a much higher confidence.

The goal then is to learn user factors Xu and artist factors Yi by minimizing a confidence weighted sum of squared errors loss function:

loss=∑u∑iCui(Pui−XuYi)2+λ(‖Xu‖2+‖Yi‖2)

Cui is the confidence that we have that the user likes the artist, Pui is a binary value indicating if the user listened to the artist or not, and the λ is a basic L2 Regularizer to reduce overfitting.

To minimize the user factors, we fix the item factors constant and then take the derivative of the loss function to calculate Xu directly:

Xu=(YTCuY+λI)−1(YTCUPu)

The item factors are calculated in a similar way, and the entire thing is minimized by alternating back and forth until it converges (hence the 'Alternative Least Squares' name).

The clever part of this paper is in how it learns over all data, but only has to do work on the non-zero items. Since Pu is sparse (the negative preferences have a 0 value), 
YTCuPu 
can be easily calculated. To calculate YTCuY they note that it's equal to YTY+YT(Cu−I)Y. By setting the confidences for negative items to 1, (Cu−I) is sparse, and YTY can be precalculated for all users.

##### Optional Lab: [Build a Matrix Factorization CF Recommender in Python]

##### For those interested in performance, see https://github.com/benfred/implicit

## ALS Spark

The Spark ALS is described here:  
[Alternating Least Square (ALS) implemented in Spark](https://spark.apache.org/docs/preview/ml-collaborative-filtering.html#collaborative-filtering)

For an implementation of ALS on a different data set see this post of the [Santander Kaggle Competition](https://www.elenacuoco.com/2016/12/22/alternating-least-squares-als-spark-ml/). 

For more information and a different take look at this [page](https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/).

Lab: 
Check out the [movie_recommendation_using_ALS](movie_recommender/movie_recommendation_using_ALS.ipynb) and implement it on EMLP.

### Some Things to consider 

#### WHAT’S THE DIFFERENCE BETWEEN ASSOCIATION RULE RECS AND COLLABORATIVE RECS?

Associative rules aren’t collaborative filtering because they’re based on what’s in a single shopping basket, not what a user buys over time. Collaborative filtering looks at what users buy or rate over time.

#### LEVERS TO TUNE FOR COLLABORATIVE FILTERING
It isn’t always enough to implement the algorithm to get good recommendations. Often there are things that need to be adjusted. For example, you previously adjusted the number of overlapping users needed before you could calculate a similarity. When considering what can be adjusted, refer to this list:


* Which ratings should be used for the active user?
  * Only the positive ones?
  * Only the most recent ones?
  * How should you normalize the ratings?

* When creating similarity, how many user’s need to rate two items for the similarity to be calculated?

* Should you restrict similarities only to be added to the similarity list if they’re positive?
  
* When predicting the ratings
  * Should you use classification or regression?
  * Should you use a weighted average?
  

* When returning the recommendations, should you return all of them or only the ones with good predictions (that is, prediction above a threshold).

And there are so many more that building a recommender may never be done.  